In [1]:
import os
import sys
sys.path.insert(0, '..')

from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt
from decouple import config
import json

from authenticate_service_account import *


## Code for retriving data from GCP

In [2]:
# client = main()

# # query = '''SELECT * FROM `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
# # WHERE Timestamp BETWEEN DATETIME("2023-08-17")
# # AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY) and Leg_0_StopsCount is not null'''

# query = '''Select * from `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
# WHERE Timestamp BETWEEN DATETIME("2023-08-17")AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY)
# AND CabinClass = 'ECONOMY'
# ORDER BY RAND()
# LIMIT 200000
# '''

# query_job = client.query(query)


# results = query_job.result()

# the_data = results.to_dataframe()


## Code for local data

In [3]:
the_data = pd.read_csv('../raw_data/rand_200k_sample', low_memory=False)


In [4]:
the_data.drop("Unnamed: 0", axis=1, inplace=True)


In [45]:
the_data['Leg_1_StopsCount'].value_counts(normalize=True)


Leg_1_StopsCount
0.0    0.712586
1.0    0.253354
2.0    0.031577
3.0    0.002483
Name: proportion, dtype: float64

In [6]:
colums_subset = ['Market', 'OriginAirport', 'OriginCity', 'DestinationAirport', 'DestinationCity', 'Locale', 'Kind',
          'TripLengthNights', 'TravelType', 'TravelDistanceKm', 'TravelHorizonDays', 'ItineraryPosition', 'ItinerarySortType',
          'PriceCustomCurrencyPerPax',
          'Leg_0_StopsCount', 'Leg_1_StopsCount',
          'Leg_0_segment_0_OperatingCarrierIATA', 'Leg_0_segment_1_OperatingCarrierIATA', 'Leg_0_segment_2_OperatingCarrierIATA', 'Leg_0_segment_3_OperatingCarrierIATA',
          'Leg_1_segment_0_OperatingCarrierIATA', 'Leg_1_segment_1_OperatingCarrierIATA', 'Leg_1_segment_2_OperatingCarrierIATA', 'Leg_1_segment_3_OperatingCarrierIATA',
          'Leg_0_DurationMin', 'Leg_1_DurationMin'
]

columns_for_comparison = ['OriginAirport', 'OriginCity', 'DestinationAirport', 'DestinationCity', 'Kind',
                          'TravelHorizonDays', 'ItineraryPosition', 'ItinerarySortType',
                        'Leg_0_StopsCount', 'Leg_1_StopsCount',
                        'Leg_0_DurationMin', 'Leg_1_DurationMin',
                        'Leg_0_segment_0_DepartureTimestamp', 'Leg_0_segment_1_DepartureTimestamp', 'Leg_0_segment_2_DepartureTimestamp',
                        'Leg_1_segment_0_DepartureTimestamp', 'Leg_1_segment_1_DepartureTimestamp', 'Leg_1_segment_2_DepartureTimestamp',
                        'Leg_0_segment_0_ArrivalTimestamp', 'Leg_0_segment_1_ArrivalTimestamp', 'Leg_0_segment_2_ArrivalTimestamp',
                        'Leg_1_segment_0_ArrivalTimestamp', 'Leg_1_segment_1_ArrivalTimestamp', 'Leg_1_segment_2_ArrivalTimestamp']

all_columns = list(the_data.columns)


In [7]:
remaining_columns = list(set(all_columns) - set(colums_subset))
remaining_columns


['Leg_1_segment_0_DepartureTimestamp',
 'Leg_0_segment_1_ArrivalTimestamp',
 'Leg_1_segment_0_OriginName',
 'Leg_0_segment_3_DepartureTimestamp',
 'Leg_1_segment_1_OriginIATA',
 'Leg_1_segment_1_FlightNumber',
 'Timestamp',
 'Leg_0_segment_1_FlightNumber',
 'ThemeCityWeight',
 'Leg_1_segment_1_TravelDistanceKm',
 'Leg_1_segment_2_DestinationIATA',
 'ThemeRomanceWeight',
 'Leg_1_segment_2_ArrivalTimestamp',
 'Leg_0_segment_0_OriginIATA',
 'RedirectType',
 'Leg_1_segment_0_TravelDistanceKm',
 'PassengerChild',
 'Leg_0_segment_0_ArrivalTimestamp',
 'Leg_1_segment_2_DurationMin',
 'Leg_0_segment_2_MarketingCarrierIATA',
 'ProviderRank',
 'Leg_1_segment_2_OriginIATA',
 'Leg_0_segment_2_DestinationName',
 'UserCountryCode',
 'Leg_0_segment_0_DurationMin',
 'Leg_0_segment_1_DurationMin',
 'Leg_0_segment_0_DepartureTimestamp',
 'Leg_1_segment_2_OriginName',
 'Leg_0_segment_3_FlightNumber',
 'Currency',
 'Leg_1_segment_2_DestinationName',
 'Leg_0_segment_2_DepartureTimestamp',
 'Leg_1_segment_0

In [8]:
timestamp_columns = ['Leg_0_segment_0_DepartureTimestamp', 'Leg_0_segment_1_DepartureTimestamp', 'Leg_0_segment_2_DepartureTimestamp',
                        'Leg_1_segment_0_DepartureTimestamp', 'Leg_1_segment_1_DepartureTimestamp', 'Leg_1_segment_2_DepartureTimestamp',
                        'Leg_0_segment_0_ArrivalTimestamp', 'Leg_0_segment_1_ArrivalTimestamp', 'Leg_0_segment_2_ArrivalTimestamp',
                        'Leg_1_segment_0_ArrivalTimestamp', 'Leg_1_segment_1_ArrivalTimestamp', 'Leg_1_segment_2_ArrivalTimestamp']


In [9]:
data_subset = the_data[columns_for_comparison].copy()
data_subset


,OriginAirport,OriginCity,DestinationAirport,DestinationCity,Kind,TravelDistanceKm,TravelHorizonDays,ItineraryPosition,ItinerarySortType,Leg_0_StopsCount,...,Leg_0_segment_2_DepartureTimestamp,Leg_1_segment_0_DepartureTimestamp,Leg_1_segment_1_DepartureTimestamp,Leg_1_segment_2_DepartureTimestamp,Leg_0_segment_0_ArrivalTimestamp,Leg_0_segment_1_ArrivalTimestamp,Leg_0_segment_2_ArrivalTimestamp,Leg_1_segment_0_ArrivalTimestamp,Leg_1_segment_1_ArrivalTimestamp,Leg_1_segment_2_ArrivalTimestamp
0,SIN,SIN,KUL,KUL,ONE_WAY,296.0,6,7.0,BEST,0,...,NaN,NaN,NaN,NaN,2023-08-23 11:35:00,NaN,NaN,NaN,NaN,NaN
1,LTN,LON,CDG,PAR,RETURN,380.0,58,0.0,BEST,0,...,NaN,2023-10-15 21:55:00,NaN,NaN,2023-10-14 08:25:00,NaN,NaN,2023-10-15 22:10:00,NaN,NaN
2,BCN,BCN,MAD,MAD,ONE_WAY,485.0,43,2.0,BEST,0,...,NaN,NaN,NaN,NaN,2023-09-29 17:50:00,NaN,NaN,NaN,NaN,NaN
3,LTN,LON,IST,IST,ONE_WAY,2496.0,17,0.0,BEST,0,...,NaN,NaN,NaN,NaN,2023-09-03 12:50:00,NaN,NaN,NaN,NaN,NaN
4,MAN,MAN,DXB,DXB,RETURN,5655.0,151,0.0,BEST,0,...,NaN,2024-01-22 14:30:00,NaN,NaN,2024-01-15 19:50:00,NaN,NaN,2024-01-22 18:40:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,NRT,TYO,ASJ,ASJ,RETURN,1296.0,62,0.0,BEST,0,...,NaN,2023-10-20 13:30:00,NaN,NaN,2023-10-18 12:50:00,NaN,NaN,2023-10-20 15:40:00,NaN,NaN
199996,BOM,BOM,LBA,LBA,ONE_WAY,7295.0,20,NaN,NaN,2,...,2023-09-07 12:40:00,NaN,NaN,NaN,2023-09-07 08:10:00,2023-09-07 10:55:00,2023-09-07 12:50:00,NaN,NaN,NaN
199997,TLV,TLV,BUD,BUD,RETURN,2167.0,45,NaN,NaN,0,...,NaN,2023-10-05 23:30:00,NaN,NaN,2023-10-01 09:05:00,NaN,NaN,2023-10-06 03:40:00,NaN,NaN
199998,FCO,ROM,KBL,KBL,RETURN,4952.0,21,0.0,CHEAPEST,1,...,NaN,2023-10-10 08:20:00,2023-10-10 21:30:00,2023-10-11 06:15:00,2023-09-07 20:50:00,2023-09-08 06:30:00,NaN,2023-10-10 11:05:00,2023-10-11 02:05:00,2023-10-11 07:35:00


In [10]:
def strip_date_keep_time(df, list_of_columns):
    for i in list_of_columns:
        df[i] = df[i].str.slice(10)

    return df


In [11]:
striped_time_df = strip_date_keep_time(data_subset, timestamp_columns)


In [12]:
striped_time_df


,OriginAirport,OriginCity,DestinationAirport,DestinationCity,Kind,TravelDistanceKm,TravelHorizonDays,ItineraryPosition,ItinerarySortType,Leg_0_StopsCount,...,Leg_0_segment_2_DepartureTimestamp,Leg_1_segment_0_DepartureTimestamp,Leg_1_segment_1_DepartureTimestamp,Leg_1_segment_2_DepartureTimestamp,Leg_0_segment_0_ArrivalTimestamp,Leg_0_segment_1_ArrivalTimestamp,Leg_0_segment_2_ArrivalTimestamp,Leg_1_segment_0_ArrivalTimestamp,Leg_1_segment_1_ArrivalTimestamp,Leg_1_segment_2_ArrivalTimestamp
0,SIN,SIN,KUL,KUL,ONE_WAY,296.0,6,7.0,BEST,0,...,NaN,NaN,NaN,NaN,11:35:00,NaN,NaN,NaN,NaN,NaN
1,LTN,LON,CDG,PAR,RETURN,380.0,58,0.0,BEST,0,...,NaN,21:55:00,NaN,NaN,08:25:00,NaN,NaN,22:10:00,NaN,NaN
2,BCN,BCN,MAD,MAD,ONE_WAY,485.0,43,2.0,BEST,0,...,NaN,NaN,NaN,NaN,17:50:00,NaN,NaN,NaN,NaN,NaN
3,LTN,LON,IST,IST,ONE_WAY,2496.0,17,0.0,BEST,0,...,NaN,NaN,NaN,NaN,12:50:00,NaN,NaN,NaN,NaN,NaN
4,MAN,MAN,DXB,DXB,RETURN,5655.0,151,0.0,BEST,0,...,NaN,14:30:00,NaN,NaN,19:50:00,NaN,NaN,18:40:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,NRT,TYO,ASJ,ASJ,RETURN,1296.0,62,0.0,BEST,0,...,NaN,13:30:00,NaN,NaN,12:50:00,NaN,NaN,15:40:00,NaN,NaN
199996,BOM,BOM,LBA,LBA,ONE_WAY,7295.0,20,NaN,NaN,2,...,12:40:00,NaN,NaN,NaN,08:10:00,10:55:00,12:50:00,NaN,NaN,NaN
199997,TLV,TLV,BUD,BUD,RETURN,2167.0,45,NaN,NaN,0,...,NaN,23:30:00,NaN,NaN,09:05:00,NaN,NaN,03:40:00,NaN,NaN
199998,FCO,ROM,KBL,KBL,RETURN,4952.0,21,0.0,CHEAPEST,1,...,NaN,08:20:00,21:30:00,06:15:00,20:50:00,06:30:00,NaN,11:05:00,02:05:00,07:35:00


## Creating OD column

In [37]:
data_subset['OD_Airport'] = data_subset['OriginAirport'] + data_subset['DestinationAirport']

data_subset['OD_City'] = data_subset['OriginCity'] + data_subset['DestinationCity']


## Top 50 OD's

In [36]:
data_subset['OD_City'].value_counts(normalize=True)[:50]


OD_City
SELOSA    0.004608
SELTYO    0.004415
SELBKK    0.003776
SELCJU    0.002707
TPETYO    0.002651
DELBOM    0.002535
SELFUK    0.002479
BOMDEL    0.002291
HKGTPE    0.002230
BLRDEL    0.002200
TYOSEL    0.002175
SELTPE    0.002012
SELDAD    0.001957
HKGTYO    0.001860
BLRBOM    0.001800
BOMBLR    0.001653
DELBLR    0.001642
LONNYC    0.001602
SELSPK    0.001566
OSASEL    0.001506
HKGBKK    0.001419
LONIST    0.001404
LONDXB    0.001394
LONBOM    0.001374
SELNHA    0.001369
BKKTYO    0.001359
SELCEB    0.001282
TPEOSA    0.001277
PNQDEL    0.001267
TPESEL    0.001262
LONDEL    0.001212
TLVLON    0.001196
HKGSEL    0.001186
BKKHKT    0.001186
SELHKG    0.001186
TLVLCA    0.001181
TYOBKK    0.001176
CJUSEL    0.001176
DELPNQ    0.001171
HKGLON    0.001171
LONBCN    0.001064
BKKSEL    0.001059
LONISB    0.001054
DELLON    0.001049
DELGOI    0.001049
BKKCNX    0.001039
DELSXR    0.001029
HKGOSA    0.001009
TLVIST    0.000988
TYOTPE    0.000983
Name: proportion, dtype: float64

## User itinierary sort preferences

In [14]:
the_data['ItinerarySortType'].value_counts(normalize=True)


ItinerarySortType
BEST                      0.835460
CHEAPEST                  0.149085
FASTEST                   0.012463
DEPARTURE_OUTBOUND_ASC    0.002890
DEPARTURE_INBOUND_ASC     0.000093
CHE                       0.000008
Name: proportion, dtype: float64

## Number of flights per airline 

In [20]:
data_subset['Leg_0_segment_0_OperatingCarrierIATA'].value_counts(normalize=True)[:30]


Leg_0_segment_0_OperatingCarrierIATA
FR    0.049538
6E    0.039131
U2    0.038053
UK    0.023979
VY    0.022252
TK    0.020787
EK    0.020227
W4    0.019605
PC    0.018245
LH    0.017219
AI    0.017209
W6    0.017062
G3    0.017031
BA    0.015869
QR    0.015173
JJ    0.013829
EY    0.010700
SG    0.010668
7C    0.010396
OZ    0.010250
SV    0.009910
CX    0.009784
RR    0.009721
TW    0.009570
QP    0.009454
FD    0.009188
AF    0.009151
LJ    0.008895
VJ    0.008868
UA    0.008806
Name: proportion, dtype: float64

## Frequency of flight choice based off itinierary position

In [21]:
data_subset["ItineraryPosition"].value_counts(normalize=True)[:10]


/var/folders/62/d7t21rj54ls65fhypjm4ryb00000gn/T/ipykernel_69038/3080863448.py:1: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  data_subset["ItineraryPosition"].value_counts(normalize=True)[:10]


ItineraryPosition
0.0     0.602608
1.0     0.155770
2.0     0.078507
3.0     0.047853
4.0     0.024568
5.0     0.017985
6.0     0.013625
7.0     0.011511
8.0     0.007251
9.0     0.004757
10.0    0.003948
Name: proportion, dtype: float64

## Number of return/one-way flights

In [22]:
data_subset['Kind'].value_counts(normalize=True)


Kind
RETURN        0.53804
ONE_WAY       0.44834
MULTI_CITY    0.01362
Name: proportion, dtype: float64

## Exploring single itiniraries

In [42]:
london_milan_mask = striped_time_df['OD_City'] == 'LONMIL'
london_ny_mask = striped_time_df['OD_City'] == 'LONNYC'
london_jhb_mask = striped_time_df['OD_City'] == "LONJNB"


In [44]:
ldn_jhb = striped_time_df[london_jhb_mask]
ldn_jhb


,OriginAirport,OriginCity,DestinationAirport,DestinationCity,Kind,TravelDistanceKm,TravelHorizonDays,ItineraryPosition,ItinerarySortType,Leg_0_StopsCount,...,Leg_1_segment_1_DepartureTimestamp,Leg_1_segment_2_DepartureTimestamp,Leg_0_segment_0_ArrivalTimestamp,Leg_0_segment_1_ArrivalTimestamp,Leg_0_segment_2_ArrivalTimestamp,Leg_1_segment_0_ArrivalTimestamp,Leg_1_segment_1_ArrivalTimestamp,Leg_1_segment_2_ArrivalTimestamp,OD_Airport,OD_City
1599,LHR,LON,JNB,JNB,RETURN,9074.0,29,1.0,BEST,0,...,NaN,NaN,10:25:00,NaN,NaN,05:00:00,NaN,NaN,LHRJNB,LONJNB
2999,LHR,LON,JNB,JNB,RETURN,9074.0,8,0.0,BEST,0,...,NaN,NaN,10:25:00,NaN,NaN,05:00:00,NaN,NaN,LHRJNB,LONJNB
5484,LHR,LON,JNB,JNB,RETURN,9074.0,43,2.0,CHEAPEST,0,...,NaN,NaN,10:25:00,NaN,NaN,05:00:00,NaN,NaN,LHRJNB,LONJNB
10501,LHR,LON,JNB,JNB,RETURN,9074.0,10,4.0,BEST,1,...,13:20:00,NaN,08:55:00,21:20:00,NaN,10:50:00,13:45:00,NaN,LHRJNB,LONJNB
18929,LHR,LON,JNB,JNB,RETURN,9074.0,17,2.0,BEST,0,...,NaN,NaN,07:45:00,NaN,NaN,07:30:00,NaN,NaN,LHRJNB,LONJNB
20707,LHR,LON,JNB,JNB,RETURN,9074.0,6,0.0,BEST,0,...,NaN,NaN,09:55:00,NaN,NaN,07:30:00,NaN,NaN,LHRJNB,LONJNB
23730,LHR,LON,JNB,JNB,RETURN,9074.0,27,3.0,BEST,1,...,07:35:00,NaN,08:55:00,21:20:00,NaN,05:45:00,08:00:00,NaN,LHRJNB,LONJNB
49528,LHR,LON,JNB,JNB,RETURN,9074.0,0,1.0,BEST,0,...,13:20:00,NaN,10:25:00,NaN,NaN,10:35:00,13:50:00,NaN,LHRJNB,LONJNB
53602,LHR,LON,JNB,JNB,RETURN,9074.0,66,NaN,NaN,0,...,NaN,NaN,09:15:00,NaN,NaN,07:40:00,NaN,NaN,LHRJNB,LONJNB
54763,LHR,LON,JNB,JNB,RETURN,9074.0,16,NaN,NaN,1,...,NaN,NaN,00:55:00,08:05:00,NaN,05:00:00,NaN,NaN,LHRJNB,LONJNB


In [32]:
ldn_ny = striped_time_df[london_ny_mask]
ldn_ny.sort_values('TravelDistanceKm')


,OriginAirport,OriginCity,DestinationAirport,DestinationCity,Kind,TravelDistanceKm,TravelHorizonDays,ItineraryPosition,ItinerarySortType,Leg_0_StopsCount,...,Leg_1_segment_1_DepartureTimestamp,Leg_1_segment_2_DepartureTimestamp,Leg_0_segment_0_ArrivalTimestamp,Leg_0_segment_1_ArrivalTimestamp,Leg_0_segment_2_ArrivalTimestamp,Leg_1_segment_0_ArrivalTimestamp,Leg_1_segment_1_ArrivalTimestamp,Leg_1_segment_2_ArrivalTimestamp,OD_Airport,OD_City
21331,STN,LON,SWF,NYC,RETURN,5531.0,39,0.0,CHEAPEST,1,...,06:30:00,NaN,13:40:00,17:50:00,NaN,04:40:00,10:30:00,NaN,STNSWF,LONNYC
111333,STN,LON,SWF,NYC,RETURN,5531.0,361,0.0,CHEAPEST,1,...,06:30:00,NaN,13:40:00,17:50:00,NaN,04:40:00,10:30:00,NaN,STNSWF,LONNYC
125865,STN,LON,SWF,NYC,RETURN,5531.0,29,NaN,NaN,1,...,06:30:00,NaN,13:40:00,17:50:00,NaN,04:40:00,10:30:00,NaN,STNSWF,LONNYC
55185,LHR,LON,LGA,NYC,RETURN,5537.0,163,0.0,BEST,1,...,18:20:00,NaN,15:45:00,19:08:00,NaN,17:19:00,06:30:00,NaN,LHRLGA,LONNYC
101937,LHR,LON,JFK,NYC,RETURN,5540.0,21,2.0,BEST,0,...,NaN,NaN,19:10:00,NaN,NaN,08:20:00,NaN,NaN,LHRJFK,LONNYC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99654,LGW,LON,JFK,NYC,RETURN,5568.0,132,2.0,BEST,0,...,NaN,NaN,18:15:00,NaN,NaN,08:05:00,NaN,NaN,LGWJFK,LONNYC
104538,LGW,LON,JFK,NYC,ONE_WAY,5568.0,14,0.0,BEST,0,...,NaN,NaN,15:50:00,NaN,NaN,NaN,NaN,NaN,LGWJFK,LONNYC
106205,LGW,LON,JFK,NYC,RETURN,5568.0,179,0.0,BEST,0,...,NaN,NaN,15:50:00,NaN,NaN,06:00:00,NaN,NaN,LGWJFK,LONNYC
89022,LGW,LON,JFK,NYC,ONE_WAY,5568.0,64,NaN,NaN,0,...,NaN,NaN,15:50:00,NaN,NaN,NaN,NaN,NaN,LGWJFK,LONNYC


In [25]:
ldn_ny['Leg_0_StopsCount'].isna().sum()


0

In [26]:
lonnyc = pd.DataFrame(ldn_ny.groupby(['OriginCity', 'DestinationCity', 'OriginAirport', 'DestinationAirport',
                'Leg_0_DurationMin', 'Leg_1_DurationMin'])['TravelHorizonDays'].count())


In [27]:
best_lonnyc = lonnyc.sort_values('TravelHorizonDays', ascending=False)


In [28]:
ldn_ny['Kind'].value_counts()


Kind
RETURN     291
ONE_WAY     25
Name: count, dtype: int64

In [29]:
ldn_ny['Leg_1_DurationMin'].isna().sum()


25